# はじめに

以下のキーワードはpythonの基礎的な使い方に関する内容であるため、今回は既知として扱います。

キーワード:<br>
クラス、インスタンス、メソッド、フィールド<br>
クラス継承、親クラス（スーパークラス）、子クラス（サブクラス）


※クラス、インスタンス、メソッド、フィールドという単語をご存知でない方はこちらを参照してください<br>
参考：[Python入門 クラスの仕組みとサンプル](https://itsakura.com/python-class)

※クラス継承、親クラス（スーパークラス）、子クラス（サブクラス）という単語をご存知でない方はこちらを参照してください<br>
参考：[【Python入門】クラスの継承についてやさしく解説](https://blog.codecamp.jp/python-class-2)


# Step1 変数クラス

Pythonは動的型付け言語なので普段はあまり意識しないが、変数はすべては何らかの型をもつ。<br>
※より正確には変数はすべて何らかのクラス（型）から生成されたインスタンスである。

例を見てみよう。

In [33]:
x = 1
print(type(x))

y = '1'
print(type(y))

<class 'int'>
<class 'str'>


それそれのクラスは独自のメソッドをもつ。<br>
これらを利用することで様々な操作を手軽に行うことができる。

以下はstrクラスがもつrepleceメソッドを使用している例である。

In [34]:
y = y.replace('1', '2')  # 1→2に置換
print(y)

2


ちなみにreplaceメソッドはstrクラスの独自メソッドなので、<br>
intクラスでreplaceを使用しようとしてもメソッドが存在せずエラーになる。

In [35]:
# x = x.replace(1, 2)  # メソッドが存在せずエラー
# print(x)

また、クラスによっては独自のフィールド(インスタンス変数?)に値を保持するものがある。<br>
例として配列型のデータ構造をもつlistとnumpy.arrayを比較しよう。

In [36]:
import numpy as np
np.array([1,2,3]).shape  # 配列の形状を保持するフィールド

(3,)

In [37]:
# list([1,2,3]).shape  # shapeをもたないのでエラー

既存のクラスを使用してDeZEROを作り上げることもできるが、独自に作成したクラスに便利なフィールドやメソッドを追加して今後の実装が楽にすることもできる。<br>
この考え方のもと、今回はデータを格納するクラスから作成してみよう。


In [38]:
class Variable:
    def __init__(self, data):
        self.data = data

データを格納するVariableクラスを作成した。<br>
今はまだデータを保持するフィールドしか持たないが、今後様々なフィールドやメソッドを追加していく。

ちなみに__init__は変数の初期化の際に呼ばれる特別なメソッドである。<br>
ここに初期化時に渡したい引数や実行したい処理を書く。

## 演習問題

In [39]:
'''
演習1-1
Variable型の変数を生成してみましょう。
data_1をdataに保持するVariable型の変数を作成してください。
'''
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data

data_1 = np.array(1.0)

### ↓ここに実装する↓ ###
var = Variable(data_1)
### ↑ここに実装する↑ ###


'''
演習1-2
1-1で作成したVariable型の変数から値をとりだし、data_2に渡してください。
'''

### ↓ここに実装する↓ ###
data_2 = var.data
### ↑ここに実装する↑ ###

print(data_2)  # 想定値：1.0

1.0


# Step2 関数を作成する

Step1で作成したVariableクラスを使って計算をするにはVariable用の関数を作成する必要がある。<br>


Variableを使って（numpyと同様の）計算が行えるようにいくつか関数を定義する

In [40]:
# 足し算
class Add:
    def __call__(self, input_x, input_y):
        x = input_x.data  # 1つ目のデータを取り出す
        y = input_y.data  # 2つ目のデータを取り出す
        z = x + y  # 実際の計算
        output = Variable(z)
        return output  # Variable型を返す

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
add = Add()
z = add(x,y)
print(z.data)

5.0


In [41]:
# 掛け算
class Multiple:
    def __call__(self, input_x, input_y):
        x = input_x.data  # 1つ目のデータを取り出す
        y = input_y.data  # 2つ目のデータを取り出す
        z = x * y  # 実際の計算
        output = Variable(z)
        return output  # Variable型を返す

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))
multiple = Multiple()
z = multiple(x,y)
print(z.data)

6.0


AddとMultipleを見比べると、実際の計算をおこなう行以外は共通していることが分かる。<br>
共通処理を関数毎で実装するのは無駄が多い。
そこで、親クラスとしてFunctionクラスを作成して共通処理をその中に集約する。

In [42]:
# 親クラス
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]   # 引数を可変長のタプル形式で受取る
        y = self.forward(*xs)  # 具体的な計算はforwardメソッドで行う
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

# 足し算
class Add(Function):
    def forward(self, x, y):
        return x + y

# 掛け算
class Multiple(Function):
    def forward(self, x, y):
        return x * y


x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

add = Add()
z = add(x,y)
print(z.data)  # 5.0

multiple = Multiple()
v = multiple(x,y)
print(v.data)

5.0
6.0


## 演習問題

In [49]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data

# 親クラス
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]   # 引数を可変長のタプル形式で受取る
        y = self.forward(*xs)  # 具体的な計算はforwardメソッドで行う
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

'''
演習2-1
入力値の2乗を出力するSquareクラスを実装し、変数xの2乗を出力してください。
その際以下に従ってください。
* Functionクラスを継承する
* 入力はnumpy.arrayを想定する
'''

x = Variable(np.array(3.0))

### ↓ここに実装する↓ ###
class Square(Function):
    def forward(self, x):
        return x**2
square = Square()

output = square(x)# 出力値をここに渡す
### ↑ここに実装する↑ ###

print(output.data)  # 想定値:9.0


'''
演習2-2  ★発展問題★
Variable型用の計算用の関数をお手軽に生成できるようにします。

FunctionFactoryは戻り値にVariable型用の関数を返します。
この関数を使って入力値のe^xを出力するexp関数を作成し、変数xのe^xを出力してください。

なお、入力値はnumpy.arrayを想定します。
'''
import numpy as np

def functionFactory(function):
    """
    # Abstract
    Variable型の変数を扱える関数を生成する

    # Params
    * function : object \n
    \t 実際の計算を行う関数
    
    # Return
    VariableFunctionのインスタンス
    """
    class VariableFuntion(Function):
        def __init__(self, function):
            self.function = function
            
        def forward(self, *inputs):
                y = self.function(*inputs)  # 具体的な計算はfunctionメソッドで行う
                return y
    
    variableFuntion = VariableFuntion(function)
    
    return variableFuntion


x = Variable(np.array(2.0))

### ↓ここに実装する↓ ###
exp = functionFactory(np.exp)
output = exp(x)# ここに出力値を渡す

### ↑ここに実装する↑ ###

print(np.round(output.data, 1))  # 想定値:7.4

9.0
7.4


# Step3 関数の連結

Variable型用の関数は入力も出力もVariable型の変数で統一している。<br>
これにより、関数の出力値をそのまま次の関数の入力値二することが出来る。

これ以上の説明は無いので早速演習問題を解いてみよう。

## 演習問題

In [44]:

import numpy as np
'''
演習3-1
以下を1行で計算してください
e^(x+y)*y
'''

# 親クラス
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]   # 引数を可変長のタプル形式で受取る
        y = self.forward(*xs)  # 具体的な計算はforwardメソッドで行う
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

# 足し算
class Add(Function):
    def forward(self, x, y):
        return x + y

# 掛け算
class Multiple(Function):
    def forward(self, x, y):
        return x * y

# 指数
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

x = Variable(np.array(1.0))
y = Variable(np.array(2.0))

add = Add()
multiple = Multiple()
exp = Exp()

### ↓ここに実装する↓ ###
output = multiple(exp(add(x, y)), y) # ここに1行で実装する

### ↑ここに実装する↑ ###

print(np.round(output.data, 1))  # 想定値:40.2

40.2


# Step4 数値微分

※数値微分自体の細かい説明は以下の回を参照：<br>
* 【機械学習勉強会】第３７回 ゼロから作るDeepLearning #4 3.1-4.6（前半） 発表者：Ryuichi
* 【機械学習勉強会】第３７回 ゼロから作るDeepLearning #4 3.1-4.6（後半） 発表者：Ryuichi

今回は数値微分自体の説明は復習程度にとどめる。

関数$f(x)$の$x$における微分の定義は以下である。
$$ f^′(x) = \lim_{h→0} \frac{f(x + h) − f(x)}{h} $$

これをプログラムで実装しよう。<br>
ただしコンピュータは極限を計算できないため$h$をできる限り近似して（0に近づけて）微分値を求める。<br>
このように微小区間の変化量を求める手法を**数値微分**という。

本来極限を求めるところ、微小区間で近似した値を計算するため誤差が含まれる。<br>
その誤差を減らす工夫に**中央差分近似**というテクニックがある。

![numerical_diff](./step1-4_fig/numerical_diff.png)

ちなみにテイラー展開により前方差分よりも中央差分の方が誤差が小さいことが証明できるそう。<br>
参考：[前進差分近似と中心差分近似の誤差](https://mochablog.org/diff-forward-central/)

今回は中央差分近似で実装する。

In [45]:
# 数値微分
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

# 2乗を出力する関数squareのx=2における微分を計算する
x = Variable(np.array(3.0))
square = functionFactory(lambda x : x**2)
y = square(x)
print(f'f(3)={y.data}')

print(f'f\'(3)={numerical_diff(square, x)}')

f(3)=9.0
f'(3)=6.000000000012662


## 演習問題

In [47]:
import numpy as np
'''
演習4-1
以下の関数のx=2における微分を求めてください
x*e^(x^2)
'''

# 親クラス
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]   # 引数を可変長のタプル形式で受取る
        y = self.forward(*xs)  # 具体的な計算はforwardメソッドで行う
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()

# 掛け算
class Multiple(Function):
    def forward(self, x, y):
        return x * y

# 2乗
class Square(Function):
    def forward(self, x):
        return x**2

# 指数
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

x = Variable(np.array(2.0))

multiple = Multiple()
exp = Exp()
square = Square()

### ↓ここに実装する↓ ###
# shiki = functionFactory(lambda x : multiple(x, exp(square(x))))
def fnc(x):
    return multiple(x, exp(square(x)))

output = numerical_diff(fnc, x)# ここに出力結果を渡す

### ↑ここに実装する↑ ###

print(np.round(output.data, 1))  # 想定値:491.4

491.4
